### Version compatibility check

This notebook compares the xsnow package installed in your environment with the documentation version it was written for. The helper below calls `scripts/check_docs_version.py` so you can confirm that the package and docs align before continuing.


In [ ]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path
import warnings


def _find_script() -> Path | None:
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        script = candidate / "scripts" / "check_docs_version.py"
        if script.exists():
            return script
    return None


def get_docs_version() -> tuple[str | None, str | None]:
    script_path = _find_script()
    if script_path is None:
        return None, "scripts/check_docs_version.py was not found"
    try:
        completed = subprocess.run(
            [sys.executable, str(script_path)],
            check=True,
            capture_output=True,
            text=True,
        )
    except subprocess.CalledProcessError as exc:
        output = (exc.stdout or "") + (exc.stderr or "")
        return None, output.strip() or str(exc)
    return completed.stdout.strip() or None, None


docs_version, docs_error = get_docs_version()

try:
    import xsnow
    package_version = xsnow.__version__
except Exception as exc:  # pylint: disable=broad-except
    xsnow = None  # type: ignore[assignment]
    package_version = None
    package_error = str(exc)
else:
    package_error = None

print(f"xsnow package version: {package_version if package_version else 'not installed'}")
if package_error and not package_version:
    print(f"Import error: {package_error}")

if docs_version:
    print(f"xsnow docs version: {docs_version}")
else:
    message = "xsnow docs version: unavailable"
    if docs_error:
        message += f" ({docs_error})"
    print(message)

if docs_version and package_version and docs_version != package_version:
    warnings.warn(
        "xsnow package version differs from the documentation version. "
        "Consider aligning them before executing the notebook.",
        stacklevel=2,
    )

# 04: Advanced Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/04_advanced_analysis.ipynb)

This notebook covers advanced snowpack analysis techniques including stability indices, hazard calculations, and using xsnow extensions.

## What You'll Learn

- Stability indices and their calculation
- Hazard chart calculations
- Critical crack length
- Comparing multiple locations and scenarios
- Advanced temporal analysis
- Using xsnow extensions


## Installation (For Colab Users)

Set `INSTALL_XSNOW = True` in the next cell if you need to install xsnow. When enabled you can pick `INSTALL_METHOD = "pip"` to install published packages or `INSTALL_METHOD = "dev"` to work from a local clone. The cell also installs the supporting scientific Python stack used throughout the course.


In [ ]:
import subprocess
import sys
from pathlib import Path

INSTALL_XSNOW = False  # Set to True to install or update xsnow in this environment.
INSTALL_METHOD = "pip"  # Choose "pip" for a package install, or "dev" for a developer clone.
DEV_REPO_URL = "https://gitlab.com/avacollabra/postprocessing/xsnow.git"
DEV_CLONE_DIR = Path.home() / "xsnow-dev"


def _run(cmd: list[str]) -> None:
    print(f"$ {' '.join(cmd)}")
    subprocess.check_call(cmd)


try:
    import xsnow
    print(f"xsnow {xsnow.__version__} is already available.")
except Exception as exc:  # pylint: disable=broad-except
    xsnow = None  # type: ignore[assignment]
    print(f"xsnow is not currently available: {exc}")
    if not INSTALL_XSNOW:
        print("Set INSTALL_XSNOW = True and re-run this cell to install xsnow (pip or dev clone).")
    else:
        try:
            if INSTALL_METHOD == "pip":
                _run([sys.executable, "-m", "pip", "install", "--quiet", "numpy", "pandas", "xarray", "matplotlib", "seaborn", "dask", "netcdf4"])
                _run([sys.executable, "-m", "pip", "install", "--quiet", "git+https://gitlab.com/avacollabra/postprocessing/xsnow"])
            elif INSTALL_METHOD == "dev":
                if not DEV_CLONE_DIR.exists():
                    _run(["git", "clone", DEV_REPO_URL, str(DEV_CLONE_DIR)])
                _run([sys.executable, "-m", "pip", "install", "--quiet", "-e", str(DEV_CLONE_DIR)])
            else:
                raise ValueError(f"Unsupported INSTALL_METHOD: {INSTALL_METHOD}")
        except subprocess.CalledProcessError as install_error:
            raise RuntimeError("xsnow installation command failed") from install_error
        import xsnow  # noqa: F401  # pylint: disable=import-outside-toplevel
        print(f"xsnow {xsnow.__version__} installed successfully.")
else:
    INSTALL_XSNOW = INSTALL_XSNOW  # no-op so variable is defined for later cells

In [ ]:
import xsnow
import os
import numpy as np
import matplotlib.pyplot as plt

# Load xsnow's lightweight sample time series dataset
print("Loading xsnow sample data for advanced analysis...")
print("Using xsnow.single_profile_timeseries()")

try:
    ds = xsnow.single_profile_timeseries()
    print("✅ Data loaded for advanced analysis")
    print(f"Dataset dimensions: {dict(ds.dims)}")
except Exception as e:
    print(f"⚠️ Error loading sample data: {e}")
    print("Make sure xsnow is properly installed:")
    print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    ds = None


## Part 1: Stability Indices

Stability indices help assess avalanche risk. Common ones include:
- **Skier Stability Index (SK38)**: Probability of skier-triggered avalanche
- **Stability Index**: General stability metric
- **Rutschblock Score**: Field test equivalent

**Note**: These calculations depend on specific formulas and may require xsnow extensions or custom implementation. Let's explore what's available:


In [ ]:
# Check if xsnow has built-in stability index functions
# Check for extension methods
extension_methods = [attr for attr in dir(ds) if not attr.startswith('_') and 
                    ('stability' in attr.lower() or 'hazard' in attr.lower() or 
                     'sk38' in attr.lower() or 'crack' in attr.lower())]

if extension_methods:
# Example: Simple stability proxy based on density gradient
# Compute density gradient (change in density with depth)
# Large gradients can indicate weak layers
density_gradient = ds['density'].diff(dim='layer')



## Part 2: Temperature Gradient Analysis

Temperature gradients are important for understanding metamorphism and weak layer formation:


In [ ]:
# Temperature gradient = change in temperature per unit depth
# Large gradients can lead to faceting (weak layer formation)

# Compute gradient using z coordinate
temp_gradient = ds['temperature'].diff(dim='layer') / ds.coords['z'].diff(dim='layer')

# Or use the temperature_gradient if already computed
if 'temperature_gradient' in ds.data_vars:
    tg = ds['temperature_gradient']
    
    # Find layers with high gradients (> 10 K/m often indicates faceting)
    high_gradient = tg > 10  # K/m
else:
    Temperature gradient analysis:
    
    # Compute gradient
    temp_gradient = ds['temperature'].diff(dim='layer') / ds.coords['z'].diff(dim='layer')
    
    # Or use pre-computed gradient
    if 'temperature_gradient' in ds.data_vars:
        tg = ds['temperature_gradient']
    
    # Find high-gradient layers (faceting conditions)
    high_gradient = tg > 10  # K/m threshold
    """)


## Part 3: Hazard Chart Calculations

Hazard charts plot stability indices against depth. xsnow may have extensions for this:


In [ ]:
# Check for hazard chart extension
# Look for hazard-related methods
if hasattr(ds, 'compute_hazard_chart') or hasattr(ds, 'hazard_chart'):
    # Example usage would be:
    # hc = ds.compute_hazard_chart()

# Manual example: Create a simple stability-depth plot
# Simple proxy: use inverse density as stability proxy
# (lower density = potentially weaker, but this is simplified!)
profile = ds.isel(location=0, time=-1, slope=0, realization=0)  # Last time step

if 'z' in profile.coords:
    depth = -profile.coords['z'].values
    density = profile['density'].values
    
    # Simple stability proxy (NOT a real stability index!)
    stability_proxy = 1.0 / (density + 1.0)  # Avoid division by zero
    
    fig, ax = plt.subplots(figsize=(8, 10))
    ax.plot(stability_proxy, depth, 'b-', linewidth=2)
    ax.set_xlabel('Stability Proxy (1/density)', fontsize=12)
    ax.set_ylabel('Depth (m)', fontsize=12)
    ax.set_title('Simple Stability-Depth Profile\n(Example - Not Real Stability Index)', 
                fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()
    


## Part 4: Critical Crack Length

Critical crack length is an advanced metric for assessing avalanche propagation potential:


In [ ]:
# Check for critical crack length method
if hasattr(ds, 'compute_critical_crack_length'):
    # Example: ccl = ds.compute_critical_crack_length()


## Part 5: Comparing Multiple Locations


In [ ]:
n_locations = ds.dims.get('location', 0)

if n_locations > 1:
    # Compare mean density across locations
    if 'density' in ds.data_vars:
        mean_density_by_location = ds['density'].mean(dim=['time', 'layer'])
        
        for i, loc in enumerate(ds.coords['location'].values):
    
    # Compare snow height evolution
    if 'HS' in ds.data_vars:
        for i, loc in enumerate(ds.coords['location'].values):
            hs_loc = ds['HS'].isel(location=i, slope=0, realization=0)
else:
    Comparing multiple locations:
    
    # Load multiple files
    ds = xsnow.read(['data/station1.pro', 'data/station2.pro'])
    
    # Compare statistics
    mean_density = ds['density'].mean(dim=['time', 'layer'])
    
    # Compare time series
    for loc in ds.coords['location']:
        hs = ds['HS'].sel(location=loc)
        # Plot or analyze
    """)


## Part 6: Advanced Temporal Analysis


In [ ]:
# Resample to daily (if data is hourly)
# This averages hourly data to daily
try:
    ds_daily = ds.resample(time='1D').mean()
except:

# Rolling window statistics
if 'HS' in ds.data_vars:
    hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
    
    # 7-day rolling average
    try:
        hs_7day = hs_series.rolling(time=7, center=True).mean()
    except:
    
    # Rate of change
    hs_rate = hs_series.diff(dim='time')


## Part 7: Using xsnow Extensions


In [ ]:
# Check what extensions/methods are available
methods = [m for m in dir(ds) if not m.startswith('_') and callable(getattr(ds, m, None))]

# Filter for potentially interesting methods
interesting = [m for m in methods if any(keyword in m.lower() for keyword in 
                                        ['compute', 'calculate', 'hazard', 'stability', 
                                         'crack', 'classify', 'mask'])]

if interesting:
    for m in interesting[:10]:  # Show first 10
else:
    xsnow extensions:
    
    # Extensions add methods to xsnowDataset
    # Common extensions include:
    # - Classification (mask_by_criteria, etc.)
    # - Hazard charts
    # - Stability indices
    # - Critical crack length
    
    # Check xsnow docs for:
    # - How to import extensions
    # - Available extension methods
    # - How to create custom extensions (see notebook 06)
    """)


## Part 8: Ensemble Analysis

If you have multiple realizations (ensemble runs), analyze uncertainty:


In [ ]:
n_realizations = ds.dims.get('realization', 0)

if n_realizations > 1:
    
    if 'HS' in ds.data_vars:
        # Compute statistics across realizations
        hs_mean = ds['HS'].mean(dim='realization')
        hs_std = ds['HS'].std(dim='realization')
        hs_min = ds['HS'].min(dim='realization')
        hs_max = ds['HS'].max(dim='realization')
        
else:
    Ensemble analysis:
    
    # Load multiple realizations
    ds = xsnow.read(['run1.pro', 'run2.pro', 'run3.pro'])
    # Or ensure realization dimension has multiple values
    
    # Statistics across ensemble
    mean = ds['HS'].mean(dim='realization')
    std = ds['HS'].std(dim='realization')
    min_val = ds['HS'].min(dim='realization')
    max_val = ds['HS'].max(dim='realization')
    
    # Plot with uncertainty bands
    # plt.fill_between(times, min_val, max_val, alpha=0.3)
    """)


## Summary

✅ **What we learned:**

1. **Stability indices**: Methods for assessing avalanche risk (may require extensions)
2. **Temperature gradients**: Important for understanding weak layer formation
3. **Hazard charts**: Visualizing stability vs depth
4. **Critical crack length**: Advanced propagation metric
5. **Multi-location comparison**: Analyzing spatial patterns
6. **Temporal analysis**: Resampling, rolling windows, rates of change
7. **Extensions**: Using xsnow's extension system
8. **Ensemble analysis**: Working with multiple realizations

## Key Advanced Techniques

- **Resampling**: `ds.resample(time='1D').mean()` for temporal aggregation
- **Rolling windows**: `ds.rolling(time=7).mean()` for smoothing
- **Ensemble stats**: `.mean(dim='realization')`, `.std(dim='realization')`
- **Extensions**: Check xsnow docs for available extension methods

## Next Steps

Ready to work with your own data? Move on to:
- **05_working_with_custom_data.ipynb**: Load and analyze your own files

Or learn to extend xsnow:
- **06_extending_xsnow.ipynb**: Create custom analysis functions

## Exercises

1. Compute temperature gradient and identify layers with gradients > 10 K/m
2. Resample hourly data to daily and compare statistics
3. If you have multiple locations, compare their mean density profiles
4. Calculate a 7-day rolling average of snow height
5. Check xsnow documentation for available extensions and try one
